In [1]:
import pandas as pd
import cx_Oracle
import pickle

In [2]:
ip = '172.16.80.74'
port = '1521'
SID = 'RPTDB'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)
connection = cx_Oracle.connect('ANLYUSR', 'ANLYUSR$', dsn_tns)

In [3]:
CIBIL = ['TBL_TRN_MB_CIB_ACCOUNT_SEGMENT','TBL_TRN_MB_CIB_ADDRESSLIST','TBL_TRN_MB_CIB_EMPLOYMENT_SEG','TBL_TRN_MB_CIB_RESPONSE_SCORE','TBL_TRN_MB_CIB_PHONELIST','TBL_TRN_MB_CIB_ID_SEGMENT','TBL_TRN_MB_CIB_ENQUIRYLIST','TBL_TRN_MB_CIB_NAMEHEADER','TBL_TRN_MB_CIB_ACCOUNTS_DETAIL']

In [4]:
for i in CIBIL:
    query = "SELECT TB3.PROPNO,sum(status) as {} \
    FROM \
    (SELECT TB1.PROPNO,decode(TB2.PROSPECT_NO,NULL,0,1) as status \
    FROM \
    (SELECT PROPNO \
    FROM TBL_TRN_CUSTOMER_DETAILS CUS \
    WHERE CUS.AGMT_DATE > TO_DATE('31-OCT-18', 'DD-MON-YY') \
    AND CUS.PORTFOLIO='CD') TB1 LEFT OUTER JOIN \
    (SELECT PROSPECT_NO FROM BIUSR.{} \
    WHERE CREATED_DATE > TO_DATE('31-OCT-18', 'DD-MON-YY') \
    AND SUBSTR(PROSPECT_NO,5,2) = 'CD') TB2 \
    ON TB1.PROPNO = TB2.PROSPECT_NO) TB3 \
    GROUP BY TB3.PROPNO".format(i,i)

    Data = pd.read_sql(query, con=connection, chunksize=10000)
    
    data = pd.concat([chunk for chunk in Data])
    
    if i == 'TBL_TRN_MB_CIB_ACCOUNT_SEGMENT':
        DF_TBL_TRN_MB_CIB_ACCOUNT_SEGMENT = data
    elif i == 'TBL_TRN_MB_CIB_ADDRESSLIST':
        DF_TBL_TRN_MB_CIB_ADDRESSLIST = data
    elif i == 'TBL_TRN_MB_CIB_EMPLOYMENT_SEG':
        DF_TBL_TRN_MB_CIB_EMPLOYMENT_SEG = data
    elif i == 'TBL_TRN_MB_CIB_RESPONSE_SCORE':
        DF_TBL_TRN_MB_CIB_RESPONSE_SCORE = data
    elif i == 'TBL_TRN_MB_CIB_PHONELIST':
        DF_TBL_TRN_MB_CIB_PHONELIST = data
    elif i == 'TBL_TRN_MB_CIB_ID_SEGMENT':
        DF_TBL_TRN_MB_CIB_ID_SEGMENT = data
    elif i == 'TBL_TRN_MB_CIB_ENQUIRYLIST':
        DF_TBL_TRN_MB_CIB_ENQUIRYLIST = data
    elif i == 'TBL_TRN_MB_CIB_NAMEHEADER':
        DF_TBL_TRN_MB_CIB_NAMEHEADER = data
    elif i == 'TBL_TRN_MB_CIB_ACCOUNTS_DETAIL':
        DF_TBL_TRN_MB_CIB_ACCOUNTS_DETAIL = data
    print("---------------------------"+i+"----DONE-------------------")
    

---------------------------TBL_TRN_MB_CIB_ACCOUNT_SEGMENT----DONE-------------------
---------------------------TBL_TRN_MB_CIB_ADDRESSLIST----DONE-------------------
---------------------------TBL_TRN_MB_CIB_EMPLOYMENT_SEG----DONE-------------------
---------------------------TBL_TRN_MB_CIB_RESPONSE_SCORE----DONE-------------------
---------------------------TBL_TRN_MB_CIB_PHONELIST----DONE-------------------
---------------------------TBL_TRN_MB_CIB_ID_SEGMENT----DONE-------------------
---------------------------TBL_TRN_MB_CIB_ENQUIRYLIST----DONE-------------------
---------------------------TBL_TRN_MB_CIB_NAMEHEADER----DONE-------------------
---------------------------TBL_TRN_MB_CIB_ACCOUNTS_DETAIL----DONE-------------------


In [5]:
DF_Data = pd.merge(DF_TBL_TRN_MB_CIB_ACCOUNT_SEGMENT, DF_TBL_TRN_MB_CIB_ADDRESSLIST, on='PROPNO', how='inner')

In [6]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_EMPLOYMENT_SEG, on='PROPNO', how='inner')

In [7]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_RESPONSE_SCORE, on='PROPNO', how='inner')

In [8]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_PHONELIST, on='PROPNO', how='inner')

In [9]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_ID_SEGMENT, on='PROPNO', how='inner')

In [10]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_ENQUIRYLIST, on='PROPNO', how='inner')

In [11]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_NAMEHEADER, on='PROPNO', how='inner')

In [12]:
DF_Data = pd.merge(DF_Data, DF_TBL_TRN_MB_CIB_ACCOUNTS_DETAIL, on='PROPNO', how='inner')

In [13]:
DF_Data.head()

,PROPNO,TBL_TRN_MB_CIB_ACCOUNT_SEGMENT,TBL_TRN_MB_CIB_ADDRESSLIST,TBL_TRN_MB_CIB_EMPLOYMENT_SEG,TBL_TRN_MB_CIB_RESPONSE_SCORE,TBL_TRN_MB_CIB_PHONELIST,TBL_TRN_MB_CIB_ID_SEGMENT,TBL_TRN_MB_CIB_ENQUIRYLIST,TBL_TRN_MB_CIB_NAMEHEADER,TBL_TRN_MB_CIB_ACCOUNTS_DETAIL
0,3000CD0043323,1,2,1,1,2,4,8,1,1
1,3000CD0048516,2,4,1,1,2,4,19,1,6
2,3000CD0049443,1,2,1,1,3,2,2,1,2
3,3000CD0049455,0,1,0,1,2,1,0,1,0
4,3000CD0049677,0,1,0,1,2,1,0,1,0


In [14]:
DF_Data.to_csv("D:\CD\CD_MB_CIBIL_6_months.csv")